In [148]:
import docx
import numpy as np
import random

# Step 1
An article about the history of Washington University is given as the information needs
to be transmitted. This WU article is in English characters, and your group will convert
the contents to binary digits using the ASCII encoding. That is, use the ASCII table and
translate this article to a digital file containing several hundreds of 1’s and 0’s.

In [149]:
# Load the document
doc = docx.Document('text.docx')
message = []
for para in doc.paragraphs:
    message.append(para.text)
text = '\n'.join(message)

# convert to ASCII
bin_digits = ''
for char in text:
    # handling special character
    if char == chr(8212):
        char = chr(45)
    # attach 8 digits for each char
    bin_digits += '0'* (8-len(bin(ord(char))[2:]))
    bin_digits += bin(ord(char))[2:]
    

`bin_digits` is the result after converting the text into binary digits

# Step 2
These ASCII encoded bits in the WU article are now divided into multiple segments, and
each segment can contain a maximum of 200 bits. The last segment may not have all 200
bits. Then, a CRC will be calculated for each segment. The CRC function to be used is
g(D) = [D16 + D12 + D5 + 1], which will generate a CRC length of 16 bits for each
segment. By attaching the 16 CRC bits, a segment becomes a frame and each frame has
216 bits (which include 200 bits of segmented data and 16 bits of CRC). (Note: this is a
traditional Layer 2 function.)

In [150]:
def get_CRC(string):
    string = list(string)
    if len(string) != 200:
        return 'incorrect input length'
    
    # initialize the dividend
    dividend = []
    for i in range(len(string)-1, -1, -1):
        if string[i] =='1':
            dividend.append(i)
    
    divisor = [16, 12, 5, 1]
    
    remainder = [199]
    while remainder[0] >= 16:
        quotient = dividend[0] - divisor[0]
        result = np.add(divisor, quotient)
        
        # get the remainder
        remainder = [x for x in dividend if x not in result] + [x for x in result if x not in dividend]
        remainder.sort(reverse=True)
        
        dividend = remainder
    
    # convert the remainder into 16bits
    CRC = []
    for i in range(15, -1, -1):
        if i in remainder:
            CRC.append('1')
        else:
            CRC.append('0')
    
    CRC = ''.join(CRC)
    return CRC

            
    

In [151]:
i = 0
new_bin_digits = ''
while 200*i < len(bin_digits):
    
    # get the CRC
    test_string = bin_digits[200*i:(200*(i+1))]
    CRC = get_CRC(test_string)

    # attach CRC at the end of each 200 digits
    new_bin_digits += test_string
    new_bin_digits += CRC
    i += 1

`new_bin_digits` is the result after segementing and attaching CRC

# Step 3
Instead of transmitting these ASCII-encoded, segmented, and CRC-attached frames, a
forward error correction (“FEC”) is first applied to each frame. There are many different
types of FEC algorithms, of which the simplest being the repetition coding. (Note: FEC is
a traditional layer 1 function.) Suppose that the transmitter needs to send bits 0110101.
As an example, the FEC may triplicate each bit to be sent. Using the above example, the
bit sequence 0110101 is FEC coded as 000111111000111000111. By sending
000111111000111000111, if any one bit is in error, e.g., 000111111000111010111
(underscored), the receiver recognizes this error since there has to be three identical bits
in a row (either 000 or 111) as each bit is triplicated at the transmitter before sending it
over the channel. Then using a simple majority reasoning, the receiver corrects the single
error bit such that the received bit sequence becomes 000111111000111000111. Note
that if one bit among the three triplicated bits is in error, the receiver can correct it. If,
however, two or three bits among the triplicated bits is in error, the receiver cannot
correct the errors. After the repetition coding is applied, each frame now contains 648
bits.

In [152]:
bin_digits_FEC = ''
for char in new_bin_digits:
    bin_digits_FEC += (char*3)

`bin_digits_FEC` is the result after FEC

# Step 4
Now insert framing bits. Use a flag pattern 0160 (i.e., 01111110) as the framing bits.
Insert the flag pattern before each FEC coded frame. After the last FEC coded frame, 
insert another flag. Be sure to bit stuff after any string of five 1s. Now the WU article is
ready to be transmitted. The transmitter at the physical layer transmits all of the flags and
frames, one bit at a time in a proper sequence.

In [153]:
def frame_stuff(frame):
    
    # i tracks the beginning of a serie of 1's
    i = 0
    while i < len(frame)-4:
        if frame[i] == '0':
            i += 1
        elif frame[i+1] == '0':
            i += 2
        elif frame[i+2] == '0':
            i += 3
        elif frame[i+3] == '0':
            i += 4
        elif frame[i+4] == '0':
            i += 5
        else:
            frame = frame[:(i+5)] + '0' + frame[(i+5):]
            i += 6
    return frame
            
        

In [154]:
flag = '01111110' 
bin_digits_stuffed = ''
i = 0

while 648*i < len(bin_digits_FEC):
    frame = bin_digits_FEC[648*i:(648*(i+1))]
    frame_stuffed = frame_stuff(frame)
    
    bin_digits_stuffed += flag
    bin_digits_stuffed += frame_stuffed
    i += 1
    
# insert another flag after the last frame
bin_digits_stuffed += flag

`bin_digits_stuffed` is the result after inserting flags and the stuffing bits

# Step 5
Assume that the WU article has been transmitted, and the receiver across the channel
receives it. Assume that each bit received at the receiver is in error with probability 0.1.
This will be done through a simulation based on random number generator. After
applying the probability of error to each bit, about 10% of the bits should be in error.
Then, correct any error by the FEC algorithm. After the FEC algorithm is applied, check
for CRC for each frame. Determine the number of frames that fail the CRC. Repeat this
step with bit error probability of 0.01.

In [163]:
# add the random error
for i in range(len(bin_digits_stuffed)):
    if random.random() <= 0.1:
        if bin_digits_stuffed[i] == '0':
            bin_digits_stuffed = bin_digits_stuffed[:i] + '1' + bin_digits_stuffed[(i+1):]
        else:
            bin_digits_stuffed = bin_digits_stuffed[:i] + '0' + bin_digits_stuffed[(i+1):]
            